In [6]:
%load_ext autoreload

%autoreload 2

import sys
sys.path.insert(1, '../enterotypes_ibm') # now we can import files from this directory too
import statevectors as ibm

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# define parameters

S = 20                              # number of species in the metacommunity
s = 16                          # number of species in each local community
LCs = 50                            # number of local communities
sites = 10000
connectance = 0.3               

SIS = [1]
SISfactor_ = 200
maxint = 0.5
minmigration = 0.4; maxmigration = 0.4
minextinction = 0.3; maxextinction = 0.3

SISfactor = np.ones(S+1, dtype=int)
SISfactor[SIS] *= SISfactor_

omega = np.random.uniform(-maxint,maxint, [S+1, S+1])
omega *= np.random.choice([0, 1], omega.shape, p=[1-connectance, connectance]) # connectance
omega[:,0] = 0.3 # growth rates

np.fill_diagonal(omega, -1.) # self-interaction
omega[0,:] = 0 # no growth of empty sites

omega_SIS = omega * SISfactor

# immigration rates
mu = np.random.uniform(minmigration,maxmigration, [S+1]); mu[0] = 0

# extinction rates
e = np.random.uniform(minextinction,maxextinction, [S+1]); e[0] = 0

initcond = np.full([S+1], int(0.66*sites/S))
initcond[0] = sites - sum(initcond[1:])

/Users/wkits/anaconda/lib/python3.5/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full([21], 330) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


Line profiling of function

In [10]:
%load_ext line_profiler

%lprun -f ibm.run_timeseries ibm.run_timeseries(omega, mu, e, initcond, SISfactor)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [11]:
import random

In [14]:
%timeit np.random.randint(1,20)
%timeit random.randint(1,20)

The slowest run took 62.25 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.43 µs per loop
The slowest run took 4.14 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.21 µs per loop


In [22]:
import time
import bisect

class WeightedRandomGenerator(object):
    # code from https://eli.thegreenplace.net/2010/01/22/weighted-random-generation-in-python
    # faster than using np random choice
    def __init__(self, weights, r=0):
        self.totals = []
        running_total = 0

        if r == 0:
            r = int(time.time())
        random.seed(r)

        for w in weights:
            running_total += w
            self.totals.append(running_total)

    def __call__(self):
        return bisect.bisect_right(self.totals, random.random() * self.totals[-1])


In [23]:
x = np.random.uniform(0,1,[20])
randomx = WeightedRandomGenerator(x)

%timeit A = randomx()

The slowest run took 7.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 916 ns per loop


In [29]:
def func_wg(x, tot):
    rnd = random.random() * tot
    return bisect.bisect_right(x, rnd)

x = np.random.uniform(0,1,[20])
tot = sum(x)

%lprun -f func_wg func_wg(x, tot)

In [53]:
x = np.linspace(0,100,5)

y = -1

%timeit bisect.bisect_right(x, y)
%timeit len(x[y >= x])

The slowest run took 8.05 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 774 ns per loop
The slowest run took 11.61 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.68 µs per loop


In [55]:
%timeit rnd = random.random() * x[-1]; bisect.bisect_right(x, rnd)
%timeit bisect.bisect_right(x, random.random() * x[-1])

The slowest run took 13.22 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 752 ns per loop
The slowest run took 6.28 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 698 ns per loop


In [59]:
%%timeit

r = np.random.uniform(0,1,50000)
for i in range(50000):
    a = r[i]

100 loops, best of 3: 9.35 ms per loop


In [60]:
%%timeit

for i in range(50000):
    a = random.random()

100 loops, best of 3: 8.55 ms per loop
